In [1]:
import read_pos_data as data
import tensorflow as tf
import numpy as np
from tensorflow.python import control_flow_ops
import time, argparse

LOADING PRETRAINED WORD2VEC MODEL... 


In [2]:
# Architecture
n_gram = 3
embedding_size = 300
n_hidden_1 = 512
n_hidden_2 = 256
n_output = len(data.tags_to_index.keys())

In [3]:
# Parameters
training_epochs = 1000
batch_size = 100
display_step = 1

In [4]:
def layer_batch_norm(x, n_out, phase_train):
    beta_init = tf.compat.v1.constant_initializer(value=0.0, dtype=tf.float32)
    gamma_init = tf.compat.v1.constant_initializer(value=1.0, dtype=tf.float32)

    beta = tf.compat.v1.get_variable("beta", [n_out], initializer=beta_init)
    gamma = tf.compat.v1.get_variable("gamma", [n_out], initializer=gamma_init)

    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
    mean, var = control_flow_ops.cond(phase_train,
        mean_var_with_update,
        lambda: (ema_mean, ema_var))

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var,
        beta, gamma, 1e-3, True)
    return tf.reshape(normed, [-1, n_out])

In [5]:
def layer(input, weight_shape, bias_shape, phase_train):
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.compat.v1.get_variable("W", weight_shape,
                        initializer=weight_init)
    b = tf.compat.v1.get_variable("b", bias_shape,
                        initializer=bias_init)
    logits = tf.matmul(input, W) + b
    return tf.nn.sigmoid(layer_batch_norm(logits, weight_shape[1], phase_train))

In [6]:
def inference(x, phase_train):
    with tf.compat.v1.variable_scope("inference"):
        with tf.compat.v1.variable_scope("hidden_1"):
            hidden_1 = layer(x, [n_gram * embedding_size, n_hidden_1], [n_hidden_1], phase_train)

        with tf.compat.v1.variable_scope("hidden_2"):
            hidden_2 = layer(hidden_1, [n_hidden_1, n_hidden_2], [n_hidden_2], phase_train)

        with tf.compat.v1.variable_scope("output"):
            output = layer(hidden_2, [n_hidden_2, n_output], [n_output], phase_train)

    return output

In [7]:
def loss(output, y):
    with tf.compat.v1.variable_scope("training"):
        xentropy = tf.nn.softmax_cross_entropy_with_logits(y, output)
        loss = tf.reduce_mean(xentropy)
        train_summary_op = tf.compat.v1.summary.scalar("train_cost", loss)
        return loss, train_summary_op

In [8]:
def training(cost, global_step):
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08,
        use_locking=False, name='Adam')
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [9]:
def evaluate(output, y):
    with tf.compat.v1.variable_scope("validation"):
        correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        val_summary_op = tf.compat.v1.summary.scalar("validation_error", (1.0 - accuracy))
        return accuracy, val_summary_op

In [10]:
print("Using a %d-gram model" % n_gram)

Using a 3-gram model


In [11]:
data.train.prepare_n_gram(n_gram)
data.test.prepare_n_gram(n_gram)

In [ ]:
with tf.Graph().as_default():

    with tf.compat.v1.variable_scope("pos_tagger"):

        x = tf.compat.v1.placeholder("float", [None, n_gram * embedding_size])
        y = tf.compat.v1.placeholder("float", [None, n_output])
        phase_train = tf.compat.v1.placeholder(tf.bool)

        output = inference(x, phase_train)

        cost, train_summary_op = loss(output, y)

        global_step = tf.Variable(0, name='global_step', trainable=False)

        train_op = training(cost, global_step)

        eval_op, val_summary_op = evaluate(output, y)

        summary_op = tf.compat.v1.summary.merge_all()

        saver = tf.compat.v1.train.Saver(max_to_keep=50)

        sess = tf.compat.v1.Session()

        train_writer = tf.compat.v1.summary.FileWriter("pos_tagger=" + str(n_gram) + "-gram_logs/",
                                            graph=sess.graph)

        val_writer = tf.compat.v1.summary.FileWriter("pos_tagger=" + str(n_gram) + "-gram_logs/",
                                            graph=sess.graph)

        init_op = tf.compat.v1.global_variables_initializer()

        sess.run(init_op)
        
        prefix = ["She", "decided", "that", "it", "was", "time", "to", "leave", "home", "."]
        sentence = ["Then", "the", "woman", ",", "after", "grabbing", "her", "umbrella", ",", "went", "to", "the", "bank", "to", "deposit", "her", "cash", "."]
        test_vectors = []
        if n_gram > 1:
            for token in prefix[1-n_gram:]:
                test_vectors.append(np.frombuffer(data.db.Get(token.encode()), dtype=np.float32))
        for token in sentence:
            test_vectors.append(np.frombuffer(data.db.Get(token.encode()), dtype=np.float32)) 
            
        test_vectors = np.array(test_vectors, dtype=np.float32)
        
        counter = 0
        test_input = []
        while counter < len(test_vectors) - n_gram + 1:
            test_input.append(test_vectors[counter:counter+n_gram].flatten())
            counter += 1

        test_input = np.array(test_input, dtype=np.float32)
        
        # Training cycle
        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(len(data.train.inputs)/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                minibatch_x, minibatch_y = data.train.minibatch(batch_size)
                # Fit training using batch data
                _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, y: minibatch_y, phase_train: True})
                train_writer.add_summary(train_summary, sess.run(global_step))
                # Compute average loss
                avg_cost += new_cost/total_batch                
            # Display logs per epoch step
            if epoch % display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost =", "{:.9f}".format(avg_cost))

                val_x, val_y = data.test.minibatch(0)
                
                train_writer.add_summary(train_summary, sess.run(global_step))

                accuracy, val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: val_x, y: val_y, phase_train: False})
                val_writer.add_summary(val_summary, sess.run(global_step))
                print("Validation Error:", (1 - accuracy))
                

                test_output = sess.run(output, feed_dict={x: test_input, phase_train: False})
                tags = []
                for tag_vector in test_output:
                    index = np.argmax(tag_vector)
                    tags.append(data.index_to_tags[index])
    
                counter = 0
                while counter < len(sentence):
                    print("%s\t\t%s" % (sentence[counter], tags[counter]))
                    counter += 1
                    
                saver.save(sess, "pos_tagger=" + str(n_gram) + "-gram_logs/model-checkpoint-" + '%04d' % (epoch+1), global_step=global_step)
   

        print("Optimization Finished!")

Epoch: 0001 cost = 3.150393490
Validation Error: 0.29212653636932373
Then		``
the		RP
woman		NN
,		(
after		UH
grabbing		VBG
her		PRP
umbrella		NN
,		(
went		UH
to		TO
the		DT
bank		NN
to		TO
deposit		NN
her		PRP
cash		JJ
.		WP$
Epoch: 0002 cost = 2.966646437
Validation Error: 0.26251161098480225
Then		``
the		RP
woman		NN
,		(
after		UH
grabbing		VBG
her		PRP$
umbrella		NN
,		(
went		UH
to		TO
the		DT
bank		NN
to		TO
deposit		NN
her		PRP$
cash		JJ
.		WP$
Epoch: 0003 cost = 2.906703859
Validation Error: 0.21918445825576782
Then		``
the		RP
woman		NN
,		(
after		IN
grabbing		VBG
her		PRP$
umbrella		NN
,		(
went		VBD
to		TO
the		DT
bank		NN
to		TO
deposit		VB
her		PRP$
cash		JJ
.		WP$
Epoch: 0004 cost = 2.873595910
Validation Error: 0.17501473426818848
Then		``
the		DT
woman		NN
,		(
after		IN
grabbing		VBG
her		PRP$
umbrella		NN
,		(
went		VBD
to		TO
the		DT
bank		NN
to		TO
deposit		VB
her		PRP$
cash		JJ
.		WP$
Epoch: 0005 cost = 2.858978143
Validation Error: 0.12924426794052124
Then		R

Epoch: 0037 cost = 2.828389508
Validation Error: 0.04440140724182129
Then		RB
the		DT
woman		NN
,		,
after		IN
grabbing		VBG
her		PRP
umbrella		NN
,		,
went		VBD
to		TO
the		DT
bank		NN
to		TO
deposit		VB
her		PRP$
cash		NN
.		.
Epoch: 0038 cost = 2.828131254
Validation Error: 0.04587578773498535
Then		RB
the		DT
woman		NN
,		,
after		IN
grabbing		VBG
her		PRP$
umbrella		NN
,		,
went		VBD
to		TO
the		DT
bank		NN
to		TO
deposit		VB
her		PRP$
cash		NN
.		.
Epoch: 0039 cost = 2.828152181
Validation Error: 0.04416966438293457
Then		RB
the		DT
woman		NN
,		,
after		IN
grabbing		VBG
her		PRP$
umbrella		NN
,		,
went		VBD
to		TO
the		DT
bank		NN
to		TO
deposit		NN
her		PRP$
cash		NN
.		.
Epoch: 0040 cost = 2.828139234
Validation Error: 0.04334819316864014
Then		RB
the		DT
woman		NN
,		,
after		IN
grabbing		VBG
her		PRP$
umbrella		NN
,		,
went		VBD
to		TO
the		DT
bank		NN
to		TO
deposit		VB
her		PRP$
cash		NN
.		.
Epoch: 0041 cost = 2.827915267
Validation Error: 0.043895840644836426
Then		RB
th